<a href="https://colab.research.google.com/github/yachi0402/stock1150207/blob/main/AI_Stock_Trader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import streamlit as st
import yfinance as yf
import pandas as pd
import ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import io

# --- 1. 頁面設定 ---
st.set_page_config(
    page_title="高勝率 AI 交易訊號儀表板",
    page_icon="📈",
    layout="wide",
    initial_sidebar_state="expanded"
)

# 自訂 CSS 優化視覺
st.markdown("""
<style>
    .metric-card {
        background-color: #262730;
        padding: 15px;
        border-radius: 10px;
        border: 1px solid #464b5f;
    }
    .stAlert {
        font-weight: bold;
    }
</style>
""", unsafe_allow_html=True)

st.title("📈 股票操盤手：日K + 5分K 智能決策系統")
st.markdown("結合 **趨勢追蹤 (MACD)** 與 **動能突破 (布林+RSI)** 的複合式策略")

# --- 2. 側邊欄設定 ---
with st.sidebar:
    st.header("⚙️ 參數設定")
    ticker_input = st.text_input("輸入股票代碼", value="2330.TW")
    st.caption("例如: 2330.TW, TSLA, AAPL, 0050.TW")

    st.divider()
    st.subheader("策略微調")
    bb_window = st.slider("布林通道週期", 10, 60, 20)
    bb_std = st.slider("布林通道標準差", 1.5, 3.0, 2.0)
    rsi_window = st.slider("RSI 週期", 7, 21, 14)

    st.divider()
    run_btn = st.button("🚀 開始分析", type="primary")

    st.info("""
    **策略邏輯：**
    1. **日K MACD**：確認大趨勢多空。
    2. **5分K 布林**：抓價格突破上軌。
    3. **5分K RSI**：確認動能 > 50。
    4. **SAR**：移動停利點位。
    """)

# --- 3. 核心函數 ---

def get_data(ticker, period, interval):
    """取得並清理 yfinance 資料"""
    try:
        # 下載資料
        df = yf.download(ticker, period=period, interval=interval, progress=False)

        if df.empty:
            return None

        # 處理 yfinance 新版 MultiIndex Columns 問題
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        df = df.reset_index()
        # 統一時間欄位名稱
        date_col = 'Date' if 'Date' in df.columns else 'Datetime'
        df.rename(columns={date_col: 'Datetime'}, inplace=True)
        df.set_index('Datetime', inplace=True)

        return df
    except Exception as e:
        st.error(f"資料讀取錯誤: {e}")
        return None

def calculate_indicators(df, bb_window, bb_std, rsi_window):
    """計算技術指標"""
    df = df.copy()

    # 1. MACD
    macd = ta.trend.MACD(close=df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()
    df['MACD_Hist'] = macd.macd_diff()

    # 2. RSI
    df['RSI'] = ta.momentum.RSIIndicator(close=df['Close'], window=rsi_window).rsi()

    # 3. Bollinger Bands
    indicator_bb = ta.volatility.BollingerBands(close=df['Close'], window=bb_window, window_dev=bb_std)
    df['BB_High'] = indicator_bb.bollinger_hband()
    df['BB_Low'] = indicator_bb.bollinger_lband()
    df['BB_Mid'] = indicator_bb.bollinger_mavg()

    # 4. SAR
    df['SAR'] = ta.trend.PSARIndicator(df['High'], df['Low'], df['Close'], step=0.02, max_step=0.2).psar()

    # 5. Volume MA
    df['Vol_MA5'] = df['Volume'].rolling(window=5).mean()

    return df

def generate_report(ticker, day_trend, checks, last_close, last_sar, strategy_msg):
    """生成文字版分析報告"""
    now_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    report = f"""
========================================
【AI 股票交易分析報告】
標的代碼: {ticker}
分析時間: {now_str}
========================================

1. 趨勢分析 (日線級別)
----------------------------------------
趨勢方向: {'多頭 🐂' if day_trend['bullish'] else '空頭/盤整 🐻'}
MACD 柱狀體: {day_trend['hist']:.4f}

2. 戰術訊號 (5分K 短線)
----------------------------------------
目前價格: {last_close:.2f}
停損參考 (SAR): {last_sar:.2f}

訊號檢核:
[ {'✅' if checks['布林突破'] else ' '} ] 布林通道突破
[ {'✅' if checks['成交量放大'] else ' '} ] 成交量大於5均量
[ {'✅' if checks['RSI > 50'] else ' '} ] RSI 動能強 (數值: {checks['RSI_Value']:.2f})
[ {'✅' if checks['站上 SAR'] else ' '} ] 價格站上 SAR

3. 總結建議
----------------------------------------
{strategy_msg}

========================================
注意：本報告僅供程式交易邏輯驗證，不構成投資建議。
    """
    return report

def plot_interactive_chart(df, ticker):
    """繪製 Plotly 互動圖表"""
    # 限制顯示最後 100 根 K 線，避免圖表太擠
    plot_df = df.tail(100)

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.05, row_heights=[0.7, 0.3],
                        subplot_titles=(f'{ticker} 5分K 價格與布林通道', '成交量與 RSI'))

    # K 線圖
    fig.add_trace(go.Candlestick(x=plot_df.index,
                                 open=plot_df['Open'], high=plot_df['High'],
                                 low=plot_df['Low'], close=plot_df['Close'],
                                 name='K線'), row=1, col=1)

    # 布林通道與 SAR
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['BB_High'], line=dict(color='rgba(0, 100, 255, 0.5)', width=1), name='布林上軌'), row=1, col=1)
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['BB_Low'], line=dict(color='rgba(0, 100, 255, 0.5)', width=1), name='布林下軌'), row=1, col=1)
    fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df['SAR'], mode='markers', marker=dict(color='orange', size=4), name='SAR 停損點'), row=1, col=1)

    # 成交量
    colors = ['green' if c >= o else 'red' for c, o in zip(plot_df['Close'], plot_df['Open'])]
    fig.add_trace(go.Bar(x=plot_df.index, y=plot_df['Volume'], marker_color=colors, name='成交量'), row=2, col=1)

    # RSI 線 (可選，這裡畫在成交量上方或獨立)
    # 為了簡化，這裡僅展示成交量，RSI 數值已在儀表板顯示

    fig.update_layout(xaxis_rangeslider_visible=False, height=600,
                      template="plotly_dark", margin=dict(l=50, r=50, t=50, b=50))

    return fig

# --- 4. 主程式邏輯 ---

if run_btn or ticker_input:
    with st.spinner('正在連線 Yahoo Finance 取得數據並運算中...'):
        # 1. 取得資料
        day_df = get_data(ticker_input, "1y", "1d")
        five_min_df = get_data(ticker_input, "5d", "5m")

    if day_df is None or five_min_df is None:
        st.error(f"❌ 無法取得 {ticker_input} 的資料。請確認代碼是否正確，或是否為開盤時間。")
    else:
        # 2. 計算指標
        day_df = calculate_indicators(day_df, bb_window, bb_std, rsi_window)
        five_min_df = calculate_indicators(five_min_df, bb_window, bb_std, rsi_window)

        # 3. 提取關鍵數據
        last_day = day_df.iloc[-1]
        last_5m = five_min_df.iloc[-1]

        # --- 策略判定 ---

        # 條件 1: 日 K 趨勢 (MACD 柱狀體 > 0 或 DIF > 0)
        day_trend_bullish = last_day['MACD_Hist'] > 0 or last_day['MACD'] > 0

        # 條件 2: 5分K 布林帶突破
        bb_breakout = last_5m['Close'] > last_5m['BB_High']

        # 條件 3: 5分K 爆量
        volume_spike = last_5m['Volume'] > last_5m['Vol_MA5']

        # 條件 4: RSI 動能
        rsi_bullish = last_5m['RSI'] > 50

        # 條件 5: SAR 翻多
        sar_bullish = last_5m['Close'] > last_5m['SAR']

        # 彙整檢查結果
        checks = {
            "布林突破": bb_breakout,
            "成交量放大": volume_spike,
            "RSI > 50": rsi_bullish,
            "站上 SAR": sar_bullish,
            "RSI_Value": last_5m['RSI']
        }

        # 決定總結訊息
        strategy_msg = ""
        strategy_color = ""

        if day_trend_bullish and checks["布林突破"] and checks["RSI > 50"]:
            strategy_msg = "🚀 **強力買進訊號**：日線多頭且短線帶量突破！建議進場並以 SAR 為移動停利。"
            strategy_color = "success"
        elif day_trend_bullish and checks["站上 SAR"] and checks["RSI > 50"]:
            strategy_msg = "👀 **觀察/持有**：趨勢偏多，但尚未發生強力突破。若已持有可續抱。"
            strategy_color = "warning"
        elif not day_trend_bullish:
            strategy_msg = "🛑 **空手觀望**：日線趨勢偏弱，建議不要做多，甚至考慮反彈做空。"
            strategy_color = "error"
        else:
            strategy_msg = "⚖️ **震盪整理**：訊號不明確，建議等待更清晰的表態。"
            strategy_color = "info"

        # --- 5. UI 呈現 ---

        # 狀態橫幅
        if strategy_color == "success":
            st.success(strategy_msg)
            st.balloons()
        elif strategy_color == "warning":
            st.warning(strategy_msg)
        elif strategy_color == "error":
            st.error(strategy_msg)
        else:
            st.info(strategy_msg)

        # 儀表板佈局
        col1, col2, col3 = st.columns([1, 1, 1])

        with col1:
            st.markdown("### 1. 日線趨勢 (大方向)")
            st.metric("日 MACD 柱狀", f"{last_day['MACD_Hist']:.2f}", delta_color="normal")
            if day_trend_bullish:
                st.markdown("🟢 **多頭架構**")
            else:
                st.markdown("🔴 **空頭/盤整**")

        with col2:
            st.markdown("### 2. 短線動能 (5分K)")
            st.metric("最新收盤價", f"{last_5m['Close']:.2f}", f"{(last_5m['Close'] - five_min_df.iloc[-2]['Close']):.2f}")
            st.metric("RSI 強度", f"{last_5m['RSI']:.2f}")

        with col3:
            st.markdown("### 3. 風控點位")
            st.metric("SAR 停損價", f"{last_5m['SAR']:.2f}")
            st.metric("布林上軌壓力", f"{last_5m['BB_High']:.2f}")

        # 詳細檢核表
        with st.expander("查看詳細訊號檢核表", expanded=True):
            c1, c2, c3, c4 = st.columns(4)
            c1.markdown(f"{'✅' if checks['布林突破'] else '⬜'} **布林突破**")
            c2.markdown(f"{'✅' if checks['成交量放大'] else '⬜'} **爆量攻擊**")
            c3.markdown(f"{'✅' if checks['RSI > 50'] else '⬜'} **RSI 多方**")
            c4.markdown(f"{'✅' if checks['站上 SAR'] else '⬜'} **站上 SAR**")

        # 圖表區
        st.divider()
        st.subheader(f"📊 {ticker_input} 5分鐘 K線戰術圖")
        chart_fig = plot_interactive_chart(five_min_df, ticker_input)
        st.plotly_chart(chart_fig, use_container_width=True)

        # --- 6. 報告匯出功能 ---
        st.divider()
        st.subheader("📂 匯出分析報告")

        col_export1, col_export2 = st.columns(2)

        # 生成文字報告
        report_text = generate_report(ticker_input, {'bullish': day_trend_bullish, 'hist': last_day['MACD_Hist']}, checks, last_5m['Close'], last_5m['SAR'], strategy_msg)

        with col_export1:
            st.text_area("報告預覽", report_text, height=200)
            st.download_button(
                label="📥 下載策略分析報告 (.txt)",
                data=report_text,
                file_name=f"{ticker_input}_AI_Strategy_Report.txt",
                mime="text/plain"
            )

        with col_export2:
            st.write("### 原始數據下載")
            st.write("包含所有計算過的技術指標 (MACD, RSI, BB, SAR)")

            # 轉換 CSV
            csv = five_min_df.to_csv().encode('utf-8')
            st.download_button(
                label="📥 下載 5分K 計算數據 (.csv)",
                data=csv,
                file_name=f"{ticker_input}_5m_Indicators.csv",
                mime="text/csv",
            )

2026-02-07 08:54:34.509 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-07 08:54:34.511 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-07 08:54:34.513 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-07 08:54:34.517 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-07 08:54:34.520 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-07 08:54:34.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-07 08:54:34.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-07 08:54:34.523 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar